In [28]:
import pandas as pd
import datetime
import numpy as np

def doy(r):
    return datetime.datetime(int(r.Date[:4]), int(r.Date[5:7]), int(r.Date[8:10])).timetuple().tm_yday

In [47]:
filename='AMF_US-Ne1_BASE-BADM_7-1/AMF_US-Ne1_BASE_HR_7-1.csv'
if(1):
    r=pd.read_csv(filename,skiprows=2).iloc[:,[0,1,3,14,15,16]]
    r.iloc[:,0:2]=r.applymap(str).iloc[:,0:2]
    r['Year'] = r.TIMESTAMP_START.str[:4]
    r['Date'] = r.TIMESTAMP_START.str[:4]+"-"+r.TIMESTAMP_START.str[4:6]+"-"+r.TIMESTAMP_START.str[6:8]
    r['DOY'] = pd.Series.to_frame(r.apply(doy,axis=1))
    r['TA'][r['TA']==-9999.0]=None
    r['P'][r['P']==-9999.0]=None
    r['TS_1'][r['TS_1']==-9999.0]=None
    r['TS_2'][r['TS_2']==-9999.0]=None
    r['TS']=0.5*(r['TS_1']+r['TS_2'])
    
    gb = r.groupby(['Year','Date'])
    """tmax=gb.agg({'Year' : np.min, 'Date':np.min,'DOY':np.min,'TA' : np.max, 'P' : 'sum'})
    tmin=gb.agg({'Year' : np.min,'Date':np.min,'DOY':np.min,'TA' : np.min, 'P' : 'sum'})

    tmax.index=range(len(tmax))
    tmin.index=range(len(tmin))
    """
    temp=gb.agg({'Year' : np.min, 'Date':np.min,'DOY':np.min,'TA' : {'tmin' : np.min, 'tmax' : np.max}, 'TS' : {'tsmin' : np.min, 'tsmax' : np.max}, 'P' : 'sum'})
    temp.index=range(len(temp))
    
    temp=pd.concat([temp,temp.TA],axis=1)
    temp.columns=pd.MultiIndex.from_tuples(list(temp.columns)[:-2]+([("TA_soy","tmin"),("TA_soy","tmax")]))
    
    temp=pd.concat([temp,temp.TS],axis=1)
    temp.columns=pd.MultiIndex.from_tuples(list(temp.columns)[:-2]+([("TS_soy","tsmin"),("TS_soy","tsmax")]))
    
    temp.TA=temp.TA.clip(8.0,30.0)
    temp.TA_soy=temp.TA_soy.clip(10.0,25.0)
    temp['tavg_corn']=0.5*(temp.TA.tmin+temp.TA.tmax)
    temp['tavg_soy']=0.5*(temp.TA_soy.tmin+temp.TA_soy.tmax)
    temp['gdd_corn']=temp['tavg_corn']-8
    temp['gdd_soy']=temp['tavg_soy']-10
    
    temp.TS=temp.TS.clip(8.0,30.0)
    temp.TS_soy=temp.TS_soy.clip(10.0,25.0)
    temp['tsavg_corn']=0.5*(temp.TS.tsmin+temp.TS.tsmax)
    temp['tsavg_soy']=0.5*(temp.TS_soy.tsmin+temp.TS_soy.tsmax)
    temp['gdds_corn']=temp['tsavg_corn']-8
    temp['gdds_soy']=temp['tsavg_soy']-10
    
    print(temp.ix[335,])

C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove

Year        amin           2001
Date        amin     2001-12-02
DOY         amin            336
TA          tmin              8
            tmax          13.37
TS          tsmin             8
            tsmax             8
P           sum               0
TA_soy      tmin             10
            tmax          13.37
TS_soy      tsmin            10
            tsmax            10
tavg_corn                10.685
tavg_soy                 11.685
gdd_corn                  2.685
gdd_soy                   1.685
tsavg_corn                    8
tsavg_soy                    10
gdds_corn                     0
gdds_soy                      0
Name: 335, dtype: object


C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


In [48]:
def ameriflux_dataprocesss(filename,saveas):
    r=pd.read_csv(filename,skiprows=2).iloc[:,[0,1,3,14,15,16]]
    r.iloc[:,0:2]=r.applymap(str).iloc[:,0:2]
    r['Year'] = r.TIMESTAMP_START.str[:4]
    r['Date'] = r.TIMESTAMP_START.str[:4]+"-"+r.TIMESTAMP_START.str[4:6]+"-"+r.TIMESTAMP_START.str[6:8]
    r['DOY'] = pd.Series.to_frame(r.apply(doy,axis=1))
    r['TA'][r['TA']==-9999.0]=None
    r['P'][r['P']==-9999.0]=None
    r['TS_1'][r['TS_1']==-9999.0]=None
    r['TS_2'][r['TS_2']==-9999.0]=None
    r['TS']=0.5*(r['TS_1']+r['TS_2'])
    
    gb = r.groupby(['Year','Date'])
    """tmax=gb.agg({'Year' : np.min, 'Date':np.min,'DOY':np.min,'TA' : np.max, 'P' : 'sum'})
    tmin=gb.agg({'Year' : np.min,'Date':np.min,'DOY':np.min,'TA' : np.min, 'P' : 'sum'})

    tmax.index=range(len(tmax))
    tmin.index=range(len(tmin))
    """
    temp=gb.agg({'Year' : np.min, 'Date':np.min,'DOY':np.min,'TA' : {'tmin' : np.min, 'tmax' : np.max}, 'TS' : {'tsmin' : np.min, 'tsmax' : np.max}, 'P' : 'sum'})
    temp.index=range(len(temp))
    
    temp=pd.concat([temp,temp.TA],axis=1)
    temp.columns=pd.MultiIndex.from_tuples(list(temp.columns)[:-2]+([("TA_soy","tmin"),("TA_soy","tmax")]))
    
    temp.TA=temp.TA.clip(8.0,30.0)
    temp.TA_soy=temp.TA_soy.clip(10.0,25.0)
    temp['tavg_corn']=0.5*(temp.TA.tmin+temp.TA.tmax)
    temp['tavg_soy']=0.5*(temp.TA_soy.tmin+temp.TA_soy.tmax)
    temp['gdd_corn']=temp['tavg_corn']-8
    temp['gdd_soy']=temp['tavg_soy']-10
    
    temp=pd.concat([temp,temp.TS],axis=1)
    temp.columns=pd.MultiIndex.from_tuples(list(temp.columns)[:-2]+([("TS_soy","tsmin"),("TS_soy","tsmax")]))
    
    temp.TS=temp.TS.clip(8.0,30.0)
    temp.TS_soy=temp.TS_soy.clip(10.0,25.0)
    temp['tsavg_corn']=0.5*(temp.TS.tsmin+temp.TS.tsmax)
    temp['tsavg_soy']=0.5*(temp.TS_soy.tsmin+temp.TS_soy.tsmax)
    temp['gdds_corn']=temp['tsavg_corn']-8
    temp['gdds_soy']=temp['tsavg_soy']-10
    
    temp.to_pickle(saveas)
    return temp
ameriflux_dataprocesss('AMF_US-Ne1_BASE-BADM_7-1/AMF_US-Ne1_BASE_HR_7-1.csv',"AMF_US-Ne1_BASE-BADM_7-1/precip_temperature_data_ne1")

C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__'

Year        Date  DOY   TA         TS         P TA_soy      tavg_corn  \
      amin        amin amin tmin tmax tsmin tsmax sum   tmin tmax             
0     2001  2001-01-01    1  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
1     2001  2001-01-02    2  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
2     2001  2001-01-03    3  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
3     2001  2001-01-04    4  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4     2001  2001-01-05    5  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
5     2001  2001-01-06    6  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
6     2001  2001-01-07    7  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
7     2001  2001-01-08    8  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
8     2001  2001-01-09    9  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
9     2001  2001-01-10   10  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
10    2001  2001-01-11   11  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
11    2001  2001-01-12   12  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
12    2001  2001-01-13   13  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
13    2001  2001-01-14   14  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
14    2001  2001-01-15   15  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
15    2001  2001-01-16   16  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
16    2001  2001-01-17   17  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
17    2001  2001-01-18   18  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
18    2001  2001-01-19   19  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
19    2001  2001-01-20   20  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
20    2001  2001-01-21   21  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
21    2001  2001-01-22   22  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
22    2001  2001-01-23   23  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
23    2001  2001-01-24   24  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
24    2001  2001-01-25   25  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
25    2001  2001-01-26   26  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
26    2001  2001-01-27   27  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
27    2001  2001-01-28   28  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
28    2001  2001-01-29   29  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
29    2001  2001-01-30   30  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
...    ...         ...  ...  ...  ...   ...   ...  ..    ...  ...       ...   
4718  2013  2013-12-02  336  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4719  2013  2013-12-03  337  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4720  2013  2013-12-04  338  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4721  2013  2013-12-05  339  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4722  2013  2013-12-06  340  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4723  2013  2013-12-07  341  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4724  2013  2013-12-08  342  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4725  2013  2013-12-09  343  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4726  2013  2013-12-10  344  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4727  2013  2013-12-11  345  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4728  2013  2013-12-12  346  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4729  2013  2013-12-13  347  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4730  2013  2013-12-14  348  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4731  2013  2013-12-15  349  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4732  2013  2013-12-16  350  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4733  2013  2013-12-17  351  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4734  2013  2013-12-18  352  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4735  2013  2013-12-19  353  NaN  NaN   NaN   NaN NaN   

In [49]:
ameriflux_dataprocesss('AMF_US-Ne2_BASE-BADM_7-1/AMF_US-Ne2_BASE_HR_7-1.csv',"AMF_US-Ne2_BASE-BADM_7-1/precip_temperature_data_ne2")
ameriflux_dataprocesss('AMF_US-Ne3_BASE-BADM_7-1/AMF_US-Ne3_BASE_HR_7-1.csv',"AMF_US-Ne3_BASE-BADM_7-1/precip_temperature_data_ne3")


C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\koutilya\AppData\Local\Continuum\Anaconda3\envs\CCMS\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__'

Year        Date  DOY   TA         TS         P TA_soy      tavg_corn  \
      amin        amin amin tmin tmax tsmin tsmax sum   tmin tmax             
0     2001  2001-01-01    1  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
1     2001  2001-01-02    2  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
2     2001  2001-01-03    3  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
3     2001  2001-01-04    4  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4     2001  2001-01-05    5  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
5     2001  2001-01-06    6  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
6     2001  2001-01-07    7  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
7     2001  2001-01-08    8  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
8     2001  2001-01-09    9  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
9     2001  2001-01-10   10  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
10    2001  2001-01-11   11  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
11    2001  2001-01-12   12  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
12    2001  2001-01-13   13  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
13    2001  2001-01-14   14  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
14    2001  2001-01-15   15  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
15    2001  2001-01-16   16  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
16    2001  2001-01-17   17  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
17    2001  2001-01-18   18  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
18    2001  2001-01-19   19  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
19    2001  2001-01-20   20  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
20    2001  2001-01-21   21  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
21    2001  2001-01-22   22  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
22    2001  2001-01-23   23  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
23    2001  2001-01-24   24  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
24    2001  2001-01-25   25  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
25    2001  2001-01-26   26  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
26    2001  2001-01-27   27  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
27    2001  2001-01-28   28  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
28    2001  2001-01-29   29  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
29    2001  2001-01-30   30  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
...    ...         ...  ...  ...  ...   ...   ...  ..    ...  ...       ...   
4718  2013  2013-12-02  336  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4719  2013  2013-12-03  337  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4720  2013  2013-12-04  338  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4721  2013  2013-12-05  339  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4722  2013  2013-12-06  340  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4723  2013  2013-12-07  341  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4724  2013  2013-12-08  342  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4725  2013  2013-12-09  343  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4726  2013  2013-12-10  344  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4727  2013  2013-12-11  345  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4728  2013  2013-12-12  346  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4729  2013  2013-12-13  347  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4730  2013  2013-12-14  348  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4731  2013  2013-12-15  349  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4732  2013  2013-12-16  350  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4733  2013  2013-12-17  351  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4734  2013  2013-12-18  352  NaN  NaN   NaN   NaN NaN    NaN  NaN       NaN   
4735  2013  2013-12-19  353  NaN  NaN   NaN   NaN NaN   

# PRISM temperature data processing

In [259]:
def prism_dataprocess(filename,saveas):
    pr=pd.read_csv(filename, skiprows=10)
    pr['tmin']=5*(pr['tmin (degrees F)']-32)/9
    pr['tmax']=5*(pr['tmax (degrees F)']-32)/9
    df = pr.drop('tmin (degrees F)', 1)
    df = pr.drop('tmax (degrees F)', 1)
    pr['Year']=pr['Date'].str[:4]
    pr['DOY']=pd.Series.to_frame(pr.apply(doy,axis=1))
    pr=pr[['Year','Date','DOY','tmin','tmax']]
    #pr.columns=pd.MultiIndex.from_tuples([('Year','amin'),('Date','amin'),('DOY','amin'),('TA','tmin'),('TA','tmax')])
    #pr=pd.concat([pr,pr.TA],axis=1)
    #pr.columns=pd.MultiIndex.from_tuples(list(pr.columns)[:-2]+([("TA_soy","tmin"),("TA_soy","tmax")]))
    
    pr.tmin=pr.tmin.clip(8.0,30.0)
    pr.tmax=pr.tmax.clip(8.0,30.0)
    #pr.TA_soy=pr.TA_soy.clip(10.0,25.0)
    pr['tavg_corn']=0.5*(pr.tmin+pr.tmax)
    #pr['tavg_soy']=0.5*(pr.TA_soy.tmin+pr.TA_soy.tmax)
    pr['gdd_corn']=pr['tavg_corn']-8
    #pr['gdd_soy']=pr['tavg_soy']-10
    pr.to_pickle(saveas)
    return pr
pr=prism_dataprocess('PRISM_NE1_stable_4km_20010101_20151231_41.1651_-96.4766.csv','prism_temperature_data_ne1')
len(pr)

5478

# Hybrid data creation from Ameriflux and PRISM
Because Ameriflux has lots of -9999 values for temperature (TA), we are replacing the tmin and tmax of its dataframe with the tmin and tmax values from the prism dataframe and recalculating the tavg and gdd values accordingly

In [261]:
def hybrid_dataprocess(ameriflux_picklepath,prism_picklepath,saveas):
    af=pd.read_pickle(ameriflux_picklepath)
    pr=pd.read_pickle(prism_picklepath)
    null_indices=af.TA.tmin.isnull()
    indices=af[null_indices].index
    af.loc[indices, 'gdd_corn'] = list(pr.loc[indices, 'gdd_corn'])
    #af.to_pickle(saveas)
    return af
hybrid_dataprocess("Temperature_data/ameriflux_temp_data_ne1","Temperature_data/prism_temp_data_ne1","Temperature_data/hybrid_temperature_data_ne1")

Year        Date  DOY   TA      TA_soy      tavg_corn tavg_soy  \
      amin        amin amin tmin tmax   tmin tmax                      
0     2001  2001-01-01    1  NaN  NaN    NaN  NaN       NaN      NaN   
1     2001  2001-01-02    2  NaN  NaN    NaN  NaN       NaN      NaN   
2     2001  2001-01-03    3  NaN  NaN    NaN  NaN       NaN      NaN   
3     2001  2001-01-04    4  NaN  NaN    NaN  NaN       NaN      NaN   
4     2001  2001-01-05    5  NaN  NaN    NaN  NaN       NaN      NaN   
5     2001  2001-01-06    6  NaN  NaN    NaN  NaN       NaN      NaN   
6     2001  2001-01-07    7  NaN  NaN    NaN  NaN       NaN      NaN   
7     2001  2001-01-08    8  NaN  NaN    NaN  NaN       NaN      NaN   
8     2001  2001-01-09    9  NaN  NaN    NaN  NaN       NaN      NaN   
9     2001  2001-01-10   10  NaN  NaN    NaN  NaN       NaN      NaN   
10    2001  2001-01-11   11  NaN  NaN    NaN  NaN       NaN      NaN   
11    2001  2001-01-12   12  NaN  NaN    NaN  NaN       NaN      NaN   
12    2001  2001-01-13   13  NaN  NaN    NaN  NaN       NaN      NaN   
13    2001  2001-01-14   14  NaN  NaN    NaN  NaN       NaN      NaN   
14    2001  2001-01-15   15  NaN  NaN    NaN  NaN       NaN      NaN   
15    2001  2001-01-16   16  NaN  NaN    NaN  NaN       NaN      NaN   
16    2001  2001-01-17   17  NaN  NaN    NaN  NaN       NaN      NaN   
17    2001  2001-01-18   18  NaN  NaN    NaN  NaN       NaN      NaN   
18    2001  2001-01-19   19  NaN  NaN    NaN  NaN       NaN      NaN   
19    2001  2001-01-20   20  NaN  NaN    NaN  NaN       NaN      NaN   
20    2001  2001-01-21   21  NaN  NaN    NaN  NaN       NaN      NaN   
21    2001  2001-01-22   22  NaN  NaN    NaN  NaN       NaN      NaN   
22    2001  2001-01-23   23  NaN  NaN    NaN  NaN       NaN      NaN   
23    2001  2001-01-24   24  NaN  NaN    NaN  NaN       NaN      NaN   
24    2001  2001-01-25   25  NaN  NaN    NaN  NaN       NaN      NaN   
25    2001  2001-01-26   26  NaN  NaN    NaN  NaN       NaN      NaN   
26    2001  2001-01-27   27  NaN  NaN    NaN  NaN       NaN      NaN   
27    2001  2001-01-28   28  NaN  NaN    NaN  NaN       NaN      NaN   
28    2001  2001-01-29   29  NaN  NaN    NaN  NaN       NaN      NaN   
29    2001  2001-01-30   30  NaN  NaN    NaN  NaN       NaN      NaN   
...    ...         ...  ...  ...  ...    ...  ...       ...      ...   
4718  2013  2013-12-02  336  NaN  NaN    NaN  NaN       NaN      NaN   
4719  2013  2013-12-03  337  NaN  NaN    NaN  NaN       NaN      NaN   
4720  2013  2013-12-04  338  NaN  NaN    NaN  NaN       NaN      NaN   
4721  2013  2013-12-05  339  NaN  NaN    NaN  NaN       NaN      NaN   
4722  2013  2013-12-06  340  NaN  NaN    NaN  NaN       NaN      NaN   
4723  2013  2013-12-07  341  NaN  NaN    NaN  NaN       NaN      NaN   
4724  2013  2013-12-08  342  NaN  NaN    NaN  NaN       NaN      NaN   
4725  2013  2013-12-09  343  NaN  NaN    NaN  NaN       NaN      NaN   
4726  2013  2013-12-10  344  NaN  NaN    NaN  NaN       NaN      NaN   
4727  2013  2013-12-11  345  NaN  NaN    NaN  NaN       NaN      NaN   
4728  2013  2013-12-12  346  NaN  NaN    NaN  NaN       NaN      NaN   
4729  2013  2013-12-13  347  NaN  NaN    NaN  NaN       NaN      NaN   
4730  2013  2013-12-14  348  NaN  NaN    NaN  NaN       NaN      NaN   
4731  2013  2013-12-15  349  NaN  NaN    NaN  NaN       NaN      NaN   
4732  2013  2013-12-16  350  NaN  NaN    NaN  NaN       NaN      NaN   
4733  2013  2013-12-17  351  NaN  NaN    NaN  NaN       NaN      NaN   
4734  2013  2013-12-18  352  NaN  NaN    NaN  NaN       NaN      NaN   
4735  2013  2013-12-19  353  NaN  NaN    NaN  NaN       NaN      NaN   
4736  2013  2013-12-20  354  NaN  NaN    NaN  NaN       NaN      NaN   
4737  2013  2013-12-21  355  NaN  NaN    NaN  NaN       NaN      NaN   
4738  2013  2013-12-22  356  NaN  NaN    NaN  NaN       NaN      NaN   
4739  2013  2013-12-23  357  NaN  NaN    NaN  NaN       NaN      NaN   
4740  2013  2013-12-24  358  NaN  NaN    NaN  

# Scrach  Pad

In [159]:
p=af[af.Year.amin=="2011"]
k=list(p.gdd)
k[281:299]

[11.460000000000001,
 8.4450000000000003,
 8.3250000000000028,
 7.379999999999999,
 7.1400000000000006,
 3.5199999999999996,
 7.2050000000000001,
 4.2650000000000006,
 0.59999999999999964,
 1.6600000000000001,
 0.32499999999999929,
 1.7750000000000004,
 6.6349999999999998,
 3.7750000000000004,
 8.3599999999999994,
 8.3000000000000007,
 6.5649999999999995,
 1.1750000000000007]

In [211]:
df=pd.DataFrame({'a':[1,2,3],'b':[4,5,6]})
print(df.columns)
columns=[('c','a'),('c','b')]

df.columns=pd.MultiIndex.from_tuples(columns)
print(df.columns)

Index(['a', 'b'], dtype='object')
MultiIndex(levels=[['c'], ['a', 'b']],
           labels=[[0, 0], [0, 1]])


In [24]:
import pickle 
p=pickle.load(open("AMF_US-Ne1_BASE-BADM_7-1/precip_temperature_data_ne1","rb"))

In [27]:
p.to_csv("test.csv")